In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from torchsummary import summary

In [2]:
data = pd.read_csv(r"D:\repos\udemy\.dataset\mnist_train.csv", delimiter=',', header=None)

In [3]:
labels = data[0]
data = data.drop(columns=[0])

In [4]:
data = np.array(data)
labels = np.array(labels)

In [5]:
# The min and max values of the data are 0 and 255
# It is normalized to 0 and 1
data_norm = data/np.max(data)

## Tensor dataset and loader

In [6]:
data_tensor = torch.tensor(data_norm).float()
label_tensor = torch.tensor(labels).long()

In [7]:
data_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [8]:
label_tensor

tensor([5, 0, 4,  ..., 5, 6, 8])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data_tensor, label_tensor,train_size=0.7)

In [10]:
train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_test, y_test)

In [11]:
train_loader = DataLoader(train_data, batch_size=64, drop_last=True, shuffle=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

In [12]:
for i in test_loader:
    print(i[0].shape, i[1].shape)

torch.Size([18000, 784]) torch.Size([18000])


# Model

In [13]:
def model():
    class ann(nn.Module):
        def __init__(self, units, layers):
            super().__init__()
            # Module to store the layers
            self.layers = nn.ModuleDict()
            self.nlayers = layers
            # Input layer
            self.layers['input'] = nn.Linear(784, units)
            for i in range(layers):
                self.layers[f'layer{i}'] = nn.Linear(units, units)
                
            self.layers['output'] = nn.Linear(units, 10)

        def forward(self, x):
            x = F.relu(self.layers['input'](x))
            for i in range(self.nlayers):
                x = F.relu(self.layers[f'layer{i}'](x))
            x = self.layers['output'](x)
            return x